In [1]:
import pandas as pd
from IPython.display import HTML
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [2]:
def get_features(label:str,prefix:str) -> pd.Series:
    features = {}
    features[f'{prefix}_api_list'] = []
    features[f'{prefix}_api_params'] = []
    features[f'{prefix}_api_num'] = 0
    features[f'{prefix}_result'] = []
    try:
        label_d = eval(label)
    except:
        print(f'error in parsing {label}')
        return pd.Series(features)
    for api in label_d['relevant APIs']:
        features[f'{prefix}_api_list'].append(str((api['tool_name'],api['api_name'])))
        features[f'{prefix}_api_params'].append(str(api['required_parameters']))
        features[f'{prefix}_api_num'] += 1
    try:
        features[f'{prefix}_result'] = ','.join([res[:-3] for res in label_d['result']])
    except:
        features[f'{prefix}_result'] = None
        print(label_d['result'])
    features[f'{prefix}_api_list'] = '<br>'.join(features[f'{prefix}_api_list'])
    features[f'{prefix}_api_params'] = '<br>'.join(features[f'{prefix}_api_params'])
    # features[f'{prefix}_result'] = label_d['result']
    return pd.Series(features)

# 自定义显示函数
def custom_display(df):
    return display(HTML(df.to_html(escape=False)))

## 标准答案处理

In [3]:
dev_df = pd.read_excel('dev.xlsx')
dev_df = pd.concat([dev_df,dev_df.label.apply(get_features,args=('A',))],axis=1)
custom_display(dev_df.head().drop(columns=['label']))

['api_1的结果', 'api_2的结果', {'api_3的结果 方向': '流入', 'api_4的结果 方向': '流出'}]


## 预测结果处理

In [4]:
def predict_process(predict_str:str):
    substr = '{"relevant APIs":'
    start_index = predict_str.find(substr)

    if start_index != -1:
        result = predict_str[start_index:]
    else:
        result = ""
    return result

predict = pd.read_table('generated_predictions.txt',header=None)#.apply(lambda s:eval(s)['label'])
predict_label = predict[0].apply(lambda s:eval(s)['predict'])
predict_label = predict_label.apply(predict_process)
predict_features = predict_label.apply(get_features,args=('B',))

error in parsing 
error in parsing {"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["中光防雷"]], "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "查询当前价", "required_parameters": ["api_0的结果"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "乘法计算", "required_parameters": ["api_1的结果", "10000"], "rely_apis": ["1"], "tool_name": "数值计算"}, {"api_id": "3", "api_name": "查询成交额", "required_parameters": ["api_0的结果"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "4", "api_name": "乘法计算", "required_parameters": ["api_2的结果", "api_3的结果"], "rely_apis": ["2", "3"], "tool_name": "数值计算"}, {"api_id": "5", "api_name": "减法计算", "required_parameters": ["api_2的结果", "api_4的结果"], "rely_apis": ["2", "4"], "tool_name": "数值计算"}, {"api_id": "6", "api_name": "减法计算", "required_parameters": ["api_2的结果", "api_4": ["api_5的结果"], "rely_apis": ["2", "5"], "tool_name": "数值计算"}], "result": ["api_6的结果"]}
error in parsing 
error in parsing 
error in pars

## 合并

In [5]:
compare_df = pd.concat([dev_df.drop(columns=['label']),predict_features],axis=1)
compare_df = compare_df.reindex(columns=['query','A_api_list','B_api_list','A_api_params','B_api_params','A_api_num','B_api_num','A_result','B_result'])
custom_display(compare_df)

query  \
0                           我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模   
1                        我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?   
2                                 我现在要买1000份兴业聚惠A基金，我需要支付多少申购费？   
3                                   南方新能源C的基金经理是谁，他管理的基金总规模有多大？   
4                                汇添富添添鑫A基金近1年的最大回撤及其同类排名信息是怎样的？   
...                                                         ...   
1087  我想知道我三个月前购买的欧普股票，如果今天卖出，净赚收入是多少，我当时购买的股票价格是每股10元，股数是200股。   
1088               华安新动力的成立年限是多久，同时我想知道它的蚂蚁金选标识和最近三年的年化收益率同类排名。   
1089                                我想用10000块买安信动态A，我需要支付多少申购费用   
1090                                              今天金现代的最高价是多少？   
1091                                请告诉我华泰柏瑞质量C基金目前的管理费率和是否支持定投   

                                                                      A_api_list  \
0        [(基金查询, 查询代码), (基金查询, 查询基金经理), (基金查询, 查询基金经理年化回报率), (基金查询, 查询基金经理管理规模)]   
1                   [(基金查询, 查询代码), (基金查询, 查询基金经理), (基金查询, 查询规模), (基金查询, 查询申购费率)]   
2     [(基金查询, 查询代码), (基金查询, 查询申购费率), (基金查询, 查询单位净值), (数值计算, 乘法计算), (数值计算, 乘法计算)]   
3                             [(基金查询, 查询代码), (基金查询, 查询基金经理), (基金查询, 查询基金经理管理规模)]   
4                         [(基金查询, 查询代码), (基金查询, 查询近期最大回撤), (基金查询, 查询近期最大回撤同类排名)]   
...                                                                          ...   
1087     [(股票查询, 查询代码), (股票查询, 查询当前价), (数值计算, 乘法计算), (数值计算, 乘法计算), (数值计算, 减法计算)]   
1088     [(基金查询, 查询代码), (基金查询, 查询成立年限), (基金查询, 查询蚂蚁金选标识), (基金查询, 查询近期年化收益率同类排名)]   
1089                                [(基金查询, 查询代码), (基金查询, 查询申购费率), (数值计算, 乘法计算)]   
1090                                               [(股票查询, 查询代码), (股票查询, 查询最高价)]   
1091                            [(基金查询, 查询代码), (基金查询, 查询管理费率), (基金查询, 查询是否支持定投)]   

                                                                    B_api_list  \
0      [(基金查询, 查询代码), (基金查询, 查询基金经理), (基金查询, 查询基金经理年化回报率), (基金查询, 查询基金经理管理规模)]   
1           [(基金查询, 查询代码), (基金查询, 查询基金经理), (基金查询, 查询基金经理管理规模), (基金查询, 查询申购费率)]   
2                   [(基金查询, 查询代码), (基金查询, 查询单位净值), (数值计算, 乘法计算), (数值计算, 乘法计算)]   
3                           [(基金查询, 查询代码), (基金查询, 查询基金经理), (基金查询, 查询基金经理管理规模)]   
4                       [(基金查询, 查询代码), (基金查询, 查询近期最大回撤), (基金查询, 查询近期最大回撤同类排名)]   
...                                                                        ...   
1087  [(股票查询, 查询代码), (股票查询, 查询开盘价), (股票查询, 查询当前价), (数值计算, 减法计算), (数值计算, 乘法计算)]   
1088   [(基金查询, 查询代码), (基金查询, 查询成立年限), (基金查询, 查询蚂蚁金选标识), (基金查询, 查询近期年化收益率同类排名)]   
1089                              [(基金查询, 查询代码), (基金查询, 查询申购费率), (数值计算, 乘法计算)]   
1090                                             [(股票查询, 查询代码), (股票查询, 查询最高价)]   
1091                          [(基金查询, 查询代码), (基金查询, 查询管理费率), (基金查询, 查询是否支持定投)]   

      A_api_num  B_api_num           A_result           B_result  
0             4          4  api_1,api_2,api_3        api_2,api_3  
1             4          4  api_1,api_2,api_3        api_2,api_3  
2             5          4              api_4              api_3  
3             3          3        api_1,api_2        api_1,api_2  
4             3          3        api_1,api_2        api_1,api_2  
...         ...        ...                ...                ...  
1087          5          5              api_4              api_4  
1088          4          4  api_1,api_2,api_3  api_1,api_2,api_3  
1089          3          3              api_2              api_2  
1090          2          2              api_1              api_1  
1091          3          3        api_1,api_2        api_1,api_2  

[1092 rows x 7 columns]

In [43]:
_.to_csv('compare.csv')

In [44]:
compare_df.query('A_api_list!=B_api_list')

query  \
950  我想知道如果我昨日开始以君正的开盘价每天购买100股，然后在每天的收盘价卖出，我昨天和今天总共能赚多少钱？   

                                                                                                                                                                       A_api_list  \
950  [(股票查询, 查询代码), (股票查询, 查询开盘价), (股票查询, 查询开盘价), (股票查询, 查询收盘价), (股票查询, 查询收盘价), (数值计算, 乘法计算), (数值计算, 乘法计算), (数值计算, 乘法计算), (数值计算, 乘法计算), (数值计算, 减法计算), (数值计算, 减法计算), (数值计算, 加法计算)]   

     A_api_num A_result B_api_list  B_api_num B_result  
950         12   api_11         []          0       []